# Práca s modelmi rodiny Meta

## Úvod

V tejto lekcii sa budeme venovať:

- Preskúmaniu dvoch hlavných modelov rodiny Meta – Llama 3.1 a Llama 3.2
- Pochopeniu prípadov použitia a scenárov pre každý model
- Ukážke kódu, ktorá predstaví jedinečné vlastnosti každého modelu

## Modely rodiny Meta

V tejto lekcii sa pozrieme na 2 modely z rodiny Meta alebo „Llama Herd“ – Llama 3.1 a Llama 3.2

Tieto modely existujú v rôznych variantoch a sú dostupné na trhu modelov Github. Viac informácií o používaní Github Models na [prototypovanie s AI modelmi](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) nájdete na tomto odkaze.

Varianty modelov:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*Poznámka: Llama 3 je tiež dostupná na Github Models, ale v tejto lekcii sa jej venovať nebudeme*


## Llama 3.1

S 405 miliardami parametrov patrí Llama 3.1 medzi open source LLM modely.

Tento model je vylepšením predchádzajúcej verzie Llama 3 a prináša:

- Väčšie kontextové okno – 128k tokenov oproti 8k tokenom
- Väčší maximálny počet výstupných tokenov – 4096 oproti 2048
- Lepšiu podporu viacerých jazykov – vďaka zvýšenému počtu trénovacích tokenov

Vďaka týmto vylepšeniam dokáže Llama 3.1 zvládnuť zložitejšie prípady použitia pri tvorbe GenAI aplikácií, vrátane:
- Natívneho volania funkcií – možnosť volať externé nástroje a funkcie mimo workflow LLM
- Lepšieho výkonu pri RAG – vďaka väčšiemu kontextovému oknu
- Generovania syntetických dát – možnosť vytvárať efektívne dáta pre úlohy ako doladenie modelu


### Natívne volanie funkcií

Llama 3.1 bola doladená tak, aby bola efektívnejšia pri volaní funkcií alebo nástrojov. Má tiež dva vstavané nástroje, ktoré model dokáže rozpoznať ako potrebné použiť na základe zadania od používateľa. Tieto nástroje sú:

- **Brave Search** – Dá sa použiť na získanie aktuálnych informácií, napríklad o počasí, prostredníctvom webového vyhľadávania
- **Wolfram Alpha** – Dá sa využiť na zložitejšie matematické výpočty, takže nie je potrebné písať vlastné funkcie.

Môžete si tiež vytvoriť vlastné nástroje, ktoré môže LLM volať.

V nasledujúcom príklade kódu:

- V systémovom zadaní definujeme dostupné nástroje (brave_search, wolfram_alpha).
- Pošleme používateľský dopyt, ktorý sa pýta na počasie v konkrétnom meste.
- LLM odpovie volaním nástroja Brave Search, ktoré bude vyzerať takto: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Poznámka: Tento príklad iba vykoná volanie nástroja. Ak chcete získať výsledky, musíte si vytvoriť bezplatný účet na stránke Brave API a definovať samotnú funkciu.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Napriek tomu, že Llama 3.1 je veľký jazykový model, jedným z jej obmedzení je multimodalita. To znamená, že nedokáže pracovať s rôznymi typmi vstupov, ako sú napríklad obrázky, a poskytovať na ne odpovede. Táto schopnosť patrí medzi hlavné novinky Llama 3.2. Medzi tieto vlastnosti patria aj:

- Multimodalita – dokáže spracovať textové aj obrazové vstupy
- Varianty v malých až stredných veľkostiach (11B a 90B) – umožňujú flexibilné možnosti nasadenia,
- Iba textové varianty (1B a 3B) – umožňujú nasadenie modelu na okrajových zariadeniach alebo mobiloch a poskytujú nízku latenciu

Podpora multimodality predstavuje veľký krok v oblasti open source modelov. Nasledujúci príklad kódu prijíma ako vstup obrázok aj textový prompt, aby získal analýzu obrázka z Llama 3.2 90B.

### Multimodálna podpora s Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Učenie sa tu nekončí, pokračujte v ceste

Po dokončení tejto lekcie si pozrite našu [kolekciu o generatívnej AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) a pokračujte v rozširovaní svojich vedomostí o generatívnej umelej inteligencii!



---

**Vyhlásenie o vylúčení zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj by sa mal považovať pôvodný dokument v jeho natívnom jazyku. Pre kritické informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
